In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from cesium import featurize
from cesium.time_series import TimeSeries

In [2]:
dataset_folder = "../all"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [3]:
metadata = pd.read_csv(f"{dataset_folder}/training_set_metadata.csv")
metadata.head(5)

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [4]:
train = pd.read_csv(f"{dataset_folder}/training_set.csv")
train.head(5)

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [5]:
features_to_use = ["amplitude",
                   "maximum",
                   "median",
                   "med_err",
                   "minimum",
                   "avg_err",
                   "mean",
                   "weighted_average"]

In [6]:
groups = train.groupby(['object_id', 'passband'])

In [7]:
times = groups.apply(lambda block: block['mjd'].values).reset_index().rename(columns={0: 'seq'})
times.head()

,object_id,passband,seq
0,615,0,"[59819.1532, 59820.1047, 59821.1026, 59822.110..."
1,615,1,"[59750.4306, 59752.4147, 59767.3045, 59770.225..."
2,615,2,"[59750.4229, 59752.407, 59767.2968, 59770.2179..."
3,615,3,"[59750.4383, 59752.4224, 59767.3122, 59770.233..."
4,615,4,"[59750.445, 59752.4334, 59767.3233, 59770.2445..."


In [8]:
flux = groups.apply(lambda block: block['flux'].values).reset_index().rename(columns={0: 'seq'})
flux.head()

,object_id,passband,seq
0,615,0,"[6.878784, 39.364853000000004, -10.422381, -65..."
1,615,1,"[-816.4343259999999, -1061.4570310000001, -815..."
2,615,2,"[-544.810303, -681.858887, -548.01355, -554.90..."
3,615,3,"[-471.385529, -524.95459, -475.51605199999995,..."
4,615,4,"[-388.984985, -393.480225, -405.663818, -400.2..."


In [9]:
err = groups.apply(lambda block: block['flux_err'].values).reset_index().rename(columns={0: 'seq'})
err.head()

,object_id,passband,seq
0,615,0,"[3.6331519999999995, 3.7756190000000003, 4.172..."
1,615,1,"[5.55337, 6.472994, 5.293019, 5.875329, 6.3068..."
2,615,2,"[3.6229519999999997, 4.0412040000000005, 3.462..."
3,615,3,"[3.8012129999999997, 3.552751, 3.340643, 3.736..."
4,615,4,"[11.395031, 3.5993459999999997, 3.496113, 3.83..."


In [10]:
fsets = {}
for object_id in metadata.object_id.unique():
    fset = featurize.featurize_time_series(times=times[times.object_id==object_id].seq.values,
                                           values=flux[flux.object_id==object_id].seq.values, 
                                           errors=err[err.object_id==object_id].seq.values,
                                           features_to_use=features_to_use)
    fsets[object_id] = fset

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:35: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/n

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/pl

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:35: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/featur

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/pl

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/pl

...

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:22: RuntimeWarning: invalid value encountered in double_scalars
  return max(abs((y_max - y_med) / y_med), abs((y_med - y_min) / y_med))
/home/frankl1/.env/plasticc

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0 * covA0B0) / amp
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:266: RuntimeWarning: invalid value encountered in sqrt
  dphase[j] = np.sqrt(np.dot(np.dot(v, m), v))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/f

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:22: RuntimeWarning: invalid value encountered in double_scalars
  return max(abs((y_max - y_med) / y_med), abs((y_med - y_min) / y_med))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/fea

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:253: RuntimeWarning: invalid value encountered in sqrt
  out_dict['trend_error'] = np.sqrt(np.diag(vmodl0))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0 * covA0B0) / amp
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:266: RuntimeWarning: invalid value encountered in sqrt
  dphase[j] = np.sqrt(np.dot(np.dot(v, m), v))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:253: RuntimeWarning: invalid value encountered in sqrt
  out_dict['trend_error'] = np.sqrt(np.diag(vmodl0))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:35: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3942: R

In [11]:
df_fset = pd.concat(fsets)
df_fset

feature         amplitude       maximum      median    med_err      minimum  \
channel                 0             0           0          0            0   
615       0    121.048016    125.182808  -10.015225   3.866380  -116.913223   
          1    880.533203    660.626343 -488.057969   4.980931 -1100.440063   
          2    646.921722    611.984558 -265.686004   3.387887  -681.858887   
          3    488.190827    445.737061 -162.170944   3.388196  -530.644592   
          4    402.069122    381.953735 -103.541366   3.625775  -422.184509   
          5    400.501618    378.188141  -85.524307   7.310321  -422.815094   
713       0     14.622504     14.509829   -3.096804   2.273302   -14.735178   
          1     10.422385      9.129021   -0.561735   1.115409   -11.715749   
          2     10.298480     10.529041   -0.117977   1.062866   -10.067919   
          3     11.862454     11.330316   -0.073896   1.629945   -12.394593   
          4     11.057367      9.827934   -0.792176   2.216952   -12.286801   
          5     14.491025     14.770886   -2.463012   5.082915   -14.211164   
730       0      4.701063      5.942166    0.024093   2.020452    -3.459960   
          1      4.543095      5.693109    0.171336   1.133529    -3.393080   
          2     11.921774     20.994711    0.491790   1.114873    -2.848838   
          3     19.503951     33.572102    0.660403   1.654032    -5.435799   
          4     23.498145     41.159981    1.004354   2.364042    -5.836310   
          5     33.234935     47.310059    2.542647   5.711615   -19.159811   
745       0     10.944189     18.014029    1.056714   1.877306    -3.874349   
          1     97.931352    192.244293    0.888115   1.024166    -3.618410   
          2    111.477483    220.795212    0.424360   1.027465    -2.159753   
          3    104.097369    203.250702    1.361369   1.615655    -4.944036   
          4     99.563790    183.633118    1.270151   2.316318   -15.494463   
          5     75.881338    141.513290    2.749555   5.399205   -10.249387   
1124      0      6.067815      5.330927    0.581027   2.153805    -6.804703   
          1     19.896143     37.170177    1.154596   1.305988    -2.622109   
          2     54.378113    106.671692    0.889142   1.259263    -2.084535   
          3     71.309338    139.818405    1.014105   1.852055    -2.800270   
          4     80.071971    143.600189    1.745012   2.599503   -16.543753   
          5     60.009063    109.157585    1.973272   6.032955   -10.860540   
...                   ...           ...         ...        ...          ...   
130739978 0     37.357846     51.247578    0.566667   8.903850   -23.468113   
          1     80.459251    157.191650   -0.311068   2.719525    -3.726851   
          2     22.959684     38.534805   -1.267119   4.239197    -7.384562   
          3     17.285638     23.924580    2.458673   8.643919   -10.646696   
          4     40.893879     46.577839   -1.819064  16.586058   -35.209919   
          5    311.488880    517.602478   11.129321  37.619656  -105.375282   
130755807 0    147.049413    271.930481    3.523321  15.873405   -22.168346   
          1     28.318704     54.464066    1.253499   4.053445    -2.173342   
          2      8.846144      8.301914   -0.472750   4.895991    -9.390373   
          3    180.521552    329.695709    4.378094   9.226317   -31.347395   
          4    203.362503    363.402466    4.891232  15.665066   -43.322540   
          5    102.181900    135.327408   -0.683763  33.384613   -69.036392   
130762946 0     93.877626     52.152622    7.376702  61.334496  -135.602631   
          1     49.707789      9.115225  -18.173016  11.713305   -90.300354   
          2     54.212563     27.459307  -29.086324  10.661510   -80.965820   
          3     65.235669     60.889641  -11.305672  11.972692   -69.581696   
          4     43.384130     37.266300  -17.599372  24.236902   -49.501961   
          5    128.713696    169.916672  -11.12057

In [12]:
df_fset.columns = df_fset.columns.droplevel(1)
df_fset

feature         amplitude       maximum      median    med_err      minimum  \
615       0    121.048016    125.182808  -10.015225   3.866380  -116.913223   
          1    880.533203    660.626343 -488.057969   4.980931 -1100.440063   
          2    646.921722    611.984558 -265.686004   3.387887  -681.858887   
          3    488.190827    445.737061 -162.170944   3.388196  -530.644592   
          4    402.069122    381.953735 -103.541366   3.625775  -422.184509   
          5    400.501618    378.188141  -85.524307   7.310321  -422.815094   
713       0     14.622504     14.509829   -3.096804   2.273302   -14.735178   
          1     10.422385      9.129021   -0.561735   1.115409   -11.715749   
          2     10.298480     10.529041   -0.117977   1.062866   -10.067919   
          3     11.862454     11.330316   -0.073896   1.629945   -12.394593   
          4     11.057367      9.827934   -0.792176   2.216952   -12.286801   
          5     14.491025     14.770886   -2.463012   5.082915   -14.211164   
730       0      4.701063      5.942166    0.024093   2.020452    -3.459960   
          1      4.543095      5.693109    0.171336   1.133529    -3.393080   
          2     11.921774     20.994711    0.491790   1.114873    -2.848838   
          3     19.503951     33.572102    0.660403   1.654032    -5.435799   
          4     23.498145     41.159981    1.004354   2.364042    -5.836310   
          5     33.234935     47.310059    2.542647   5.711615   -19.159811   
745       0     10.944189     18.014029    1.056714   1.877306    -3.874349   
          1     97.931352    192.244293    0.888115   1.024166    -3.618410   
          2    111.477483    220.795212    0.424360   1.027465    -2.159753   
          3    104.097369    203.250702    1.361369   1.615655    -4.944036   
          4     99.563790    183.633118    1.270151   2.316318   -15.494463   
          5     75.881338    141.513290    2.749555   5.399205   -10.249387   
1124      0      6.067815      5.330927    0.581027   2.153805    -6.804703   
          1     19.896143     37.170177    1.154596   1.305988    -2.622109   
          2     54.378113    106.671692    0.889142   1.259263    -2.084535   
          3     71.309338    139.818405    1.014105   1.852055    -2.800270   
          4     80.071971    143.600189    1.745012   2.599503   -16.543753   
          5     60.009063    109.157585    1.973272   6.032955   -10.860540   
...                   ...           ...         ...        ...          ...   
130739978 0     37.357846     51.247578    0.566667   8.903850   -23.468113   
          1     80.459251    157.191650   -0.311068   2.719525    -3.726851   
          2     22.959684     38.534805   -1.267119   4.239197    -7.384562   
          3     17.285638     23.924580    2.458673   8.643919   -10.646696   
          4     40.893879     46.577839   -1.819064  16.586058   -35.209919   
          5    311.488880    517.602478   11.129321  37.619656  -105.375282   
130755807 0    147.049413    271.930481    3.523321  15.873405   -22.168346   
          1     28.318704     54.464066    1.253499   4.053445    -2.173342   
          2      8.846144      8.301914   -0.472750   4.895991    -9.390373   
          3    180.521552    329.695709    4.378094   9.226317   -31.347395   
          4    203.362503    363.402466    4.891232  15.665066   -43.322540   
          5    102.181900    135.327408   -0.683763  33.384613   -69.036392   
130762946 0     93.877626     52.152622    7.376702  61.334496  -135.602631   
          1     49.707789      9.115225  -18.173016  11.713305   -90.300354   
          2     54.212563     27.459307  -29.086324  10.661510   -80.965820   
          3     65.235669     60.889641  -11.305672  11.972692   -69.581696   
          4     43.384130     37.266300  -17.599372  24.236902   -49.501961   
          5    128.713696    169.916672  -11.120573  45.824883   -87.510719   
130772921 0     24.886840     37.959984   -1.68239

In [13]:
df_fset.to_csv("feature_set.csv")

In [14]:
pd.read_csv("feature_set.csv", index_col=[0, 1])

amplitude       maximum      median    med_err      minimum  \
615       0    121.048016    125.182808  -10.015225   3.866380  -116.913223   
          1    880.533203    660.626343 -488.057968   4.980931 -1100.440063   
          2    646.921722    611.984558 -265.686005   3.387887  -681.858887   
          3    488.190826    445.737061 -162.170945   3.388196  -530.644592   
          4    402.069122    381.953735 -103.541367   3.625775  -422.184509   
          5    400.501618    378.188141  -85.524307   7.310321  -422.815094   
713       0     14.622504     14.509829   -3.096805   2.273303   -14.735178   
          1     10.422385      9.129021   -0.561735   1.115409   -11.715749   
          2     10.298480     10.529041   -0.117976   1.062866   -10.067919   
          3     11.862454     11.330316   -0.073896   1.629944   -12.394593   
          4     11.057367      9.827934   -0.792176   2.216952   -12.286801   
          5     14.491025     14.770886   -2.463012   5.082915   -14.211164   
730       0      4.701063      5.942166    0.024093   2.020452    -3.459960   
          1      4.543095      5.693109    0.171336   1.133529    -3.393080   
          2     11.921774     20.994711    0.491791   1.114873    -2.848838   
          3     19.503950     33.572102    0.660403   1.654032    -5.435799   
          4     23.498145     41.159981    1.004354   2.364042    -5.836310   
          5     33.234935     47.310059    2.542647   5.711615   -19.159811   
745       0     10.944189     18.014029    1.056714   1.877306    -3.874349   
          1     97.931352    192.244293    0.888115   1.024166    -3.618410   
          2    111.477482    220.795212    0.424360   1.027466    -2.159753   
          3    104.097369    203.250702    1.361369   1.615656    -4.944036   
          4     99.563790    183.633118    1.270151   2.316318   -15.494463   
          5     75.881338    141.513290    2.749555   5.399205   -10.249387   
1124      0      6.067815      5.330927    0.581027   2.153805    -6.804703   
          1     19.896143     37.170177    1.154596   1.305988    -2.622109   
          2     54.378113    106.671692    0.889142   1.259263    -2.084535   
          3     71.309338    139.818405    1.014106   1.852055    -2.800270   
          4     80.071971    143.600189    1.745012   2.599503   -16.543753   
          5     60.009063    109.157585    1.973272   6.032955   -10.860540   
...                   ...           ...         ...        ...          ...   
130739978 0     37.357846     51.247578    0.566667   8.903850   -23.468113   
          1     80.459251    157.191650   -0.311068   2.719526    -3.726851   
          2     22.959684     38.534805   -1.267119   4.239196    -7.384562   
          3     17.285638     23.924580    2.458673   8.643919   -10.646696   
          4     40.893879     46.577839   -1.819064  16.586058   -35.209919   
          5    311.488880    517.602478   11.129321  37.619656  -105.375282   
130755807 0    147.049414    271.930481    3.523321  15.873405   -22.168346   
          1     28.318704     54.464066    1.253499   4.053445    -2.173342   
          2      8.846144      8.301914   -0.472750   4.895991    -9.390373   
          3    180.521552    329.695709    4.378094   9.226317   -31.347395   
          4    203.362503    363.402466    4.891232  15.665066   -43.322540   
          5    102.181900    135.327408   -0.683763  33.384613   -69.036392   
130762946 0     93.877627     52.152622    7.376702  61.334496  -135.602631   
          1     49.707790      9.115225  -18.173016  11.713305   -90.300354   
          2     54.212564     27.459307  -29.086324  10.661510   -80.965820   
          3     65.235669     60.889641  -11.305672  11.972692   -69.581696   
          4     43.384130     37.266300  -17.599372  24.236902   -49.501961   
          5    128.713696    169.916672  -11.120573  45.824883   -87.510719   
130772921 0     24.886840     37.959984   -1.682394   9.837848   -